In [2]:
from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr

In [3]:
load_dotenv(override=True)
openai = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [4]:
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [5]:
def push(message):
    print(f"Pushing message: {message}")
    payload = {
        "token": pushover_token,
        "user": pushover_user,
        "message": message,
    }
    response = requests.post(pushover_url, data=payload)
    print(f"Pushover response: {response.json()}")

In [6]:
push("Hello, Shree!")

Pushing message: Hello, Shree!
Pushover response: {'status': 1, 'request': 'f589d392-5dc2-4ddd-9ea0-ddf4bb4977d1'}


In [7]:
# function to record user details if they want to get in touch with me
def record_user_details(email, name="Name not provided", notes="No notes provided"):
    push(f"Recording interest from {name}, with email {email} and notes: {notes}")
    return{"recorded":"ok"}

In [8]:
# function to record a question which could not be answered by the chatbot
def record_unknown_question(question):
    push(f"Recorded question that I could not answer: {question}")
    return{"recorded":"ok"}

In [9]:
# defining the json to be created to record user details from the function created above
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
        }
    }

In [10]:
# defining the json to be created to record unknown question details from the function created above
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {"type": "object",
    "properties": {
        "question": {
            "type": "string",
            "description": "The question which couldn't be answered by the chatbot"
        }
    }
    },
    "required": ["email"],
    "additionalProperties": False
}

In [11]:
# Adding both the functions to a list called tools.
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}
]

In [12]:
# Adding a function to append data to correct json based on the tool selected or called.
def handle_tool_calls(tool_calls):
    results= []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called was {tool_name}", flush=True)

        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)

        results.append({"role":"tool", "content": json.dumps(result), "tool_call_id":tool_call.id})
    return results

In [13]:
# Another way of Adding a function to append data to correct json based on the tool selected or called.
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called was {tool_name}", flush=True)

        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role":"tool", "content": json.dumps(result), "tool_call_id":tool_call.id})
    return results

In [14]:
reader = PdfReader("me/linkedin_shree.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

with open("me/summary_shree.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "Shree Kumar Sahu"

In [15]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [16]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False
    while not done:
        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)
        finish_reason = response.choices[0].finish_reason

        if finish_reason == "tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content


In [17]:
gr.ChatInterface(chat, type="messages", title="Shree's career Chatbot", description="Ask me anything about my work experience, for example what are your key skills or what do you do?").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
